[View in Colaboratory](https://colab.research.google.com/github/RayleighKim/M10_Intro_to_Classification/blob/master/10_1_Churn_Prediction.ipynb)

# Intro to Classification 01

## Churn Prediction!

### Your name:

#### 실습목표<br>
1. Scikit-Learn 을 이용하여 Logistic Regression을 할 줄 안다.
2. Scikit-Learn 을 이용하여 Artificial Neural Network를 사용할 줄 안다.
---------------
Rayleigh Kim @ D:plus

In [0]:
'''
라이브러리들을 불러오자.
'''
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

## Data Loading & Preparation

이번 실습은 다음의 정말 중요한 과정이 생략될 것이다.

Raw_data 에서, **어떤 과정**을 거치면 굉장히 잘 정돈된 Data를 얻게 된다.

**어떤 과정이란 다음과 같다.**
1. 원래 데이터는 어떻게 생겼는가?
2. 문제 정의
    * 이탈자는 어떻게 정의할 것인가.
    * 어떤 기간의 데이터로, 어떤 기간을 예측할 것인가.
3. Data Wrangling & Feature Engineering


다음의 데이터들을 구경해보자. 로컬에서 csv파일을 엑셀로 열어도 좋다.

동시에, **(new주석)RetailChurnTemplate_FeatureEngg.xlsx** 파일을 열어서 감상하자.


    

In [0]:
'''
고객 정보 데이터
'''
user_info = pd.read_csv('https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/RetailChurn_UserInfoData.csv')
user_info.head()

In [0]:
'''
Transaction 데이터
'''

raw_data = pd.read_csv('https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/RetailChurn_Transaction.csv')
raw_data.head()

하지만 우리는 바로 아래의 데이터를 사용할 것이다.

In [0]:
churn = pd.read_csv('https://raw.githubusercontent.com/RayleighKim/Example_datasets/master/RetailChurnTemplate_FeatureEngg.csv')
'''
아래의 코드는 row를 셔플하기 위해 필요하다.
'''
churn = churn.sample(frac = 1, random_state = 2018).reset_index(drop=True)

churn.head()

In [0]:
# User는 따로 떼어 둬야 어떤 User인지 알 수 있다.
Users = churn[['UserId']]

# Unknown 투성이인 Gender 와 UserType을 버리자
churn_dropped = churn.drop(['UserId', 'Gender', 'UserType'], axis=1)
churn_dropped.head()

In [0]:
# 어딘가에 있을지 모를 NA, NaN 등을 찾자. 
churn_dropped.isna().any()

더미를 이용하여 펼치자.

컴퓨터는 A, B, C, D라고 입력되어 있어봤자 그걸 그대로 인식할 수 없다!

In [0]:
churn_dropped.head()

In [0]:
pd.get_dummies(churn_dropped[['Age','Address']]).head()

In [0]:
churn_processed = pd.concat([churn_dropped, pd.get_dummies(churn_dropped[['Age','Address']])], axis=1).drop(['Age','Address'], axis=1)

churn_processed['label'] = 1
churn_processed.loc[churn_processed['Label']=='Nonchurner', 'label'] = 0

churn_ready = churn_processed.drop('Label', axis=1)

churn_ready.head()

## Train set, Test set

시계열은 아니다!

데이터의 모양을 먼저 본다

In [0]:
churn_processed.shape

In [0]:
X_train, X_test, Y_train, Y_test = churn_ready.loc[:7000].drop('label', axis=1), \
churn_ready.loc[7001:].drop('label', axis=1),\
churn_ready.loc[:7000, 'label'],\
churn_ready.loc[7001:, 'label']

In [0]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

## Logistic Regression

In [0]:
clf = LogisticRegression()
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_train = clf.predict(X_train)

print('트레이닝 7001명 중 실제 churner의 비율 : ',Y_train.mean())
print('트레이닝 7001명에 대해서 예측된 churner의 비율 :', Y_pred_train.mean())
print('모델의 트레이닝셋위의 정확도는 : {:.2f}%'.format(clf.score(X_train, Y_train)*100))

print('--------------------------------------------')

print('테스트 2158명 중 실제 churner의 비율 : ',Y_test.mean())
print('테스트 2158명에 대해서 예측된 churner의 비율 :', Y_pred.mean())
print('모델의 테스트셋위의 정확도는 : {:.2f}%'.format(clf.score(X_test, Y_test)*100))


print('---------------------------------------------')
print(confusion_matrix(Y_test,Y_pred).T)


print('---------------------------------------------')
print('precision : True라고 예측한 것 중 진짜 True 비율')
print('recall : 진짜 True 중 True라고 예측된 것의 비율')
print(classification_report(Y_test, Y_pred, target_names=('Non-churner','churner')))
pd.DataFrame({'colnames':X_train.columns, 'coef':clf.coef_.reshape(-1,)})

## K- nearest Neighbor  Classification

In [0]:
# 아래의 n_neighbors를 바꿔가며 관찰해보자.

clf = KNeighborsClassifier(n_neighbors = 30)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_train = clf.predict(X_train)

print('트레이닝 7001명 중 실제 churner의 비율 : ',Y_train.mean())
print('트레이닝 7001명에 대해서 예측된 churner의 비율 :', Y_pred_train.mean())
print('모델의 트레이닝셋위의 정확도는 : {:.2f}'.format(clf.score(X_train, Y_train)*100))

print('--------------------------------------------')

print('테스트 2158명 중 실제 churner의 비율 : ',Y_test.mean())
print('테스트 2158명에 대해서 예측된 churner의 비율 :', Y_pred.mean())
print('모델의 테스트셋위의 정확도는 : {:.2f}'.format(clf.score(X_test, Y_test)*100))


print('---------------------------------------------')
print(confusion_matrix(Y_test,Y_pred).T)


print('---------------------------------------------')
print('precision : True라고 예측한 것 중 진짜 True 비율')
print('recall : 진짜 True 중 True라고 예측된 것의 비율')
print(classification_report(Y_test, Y_pred, target_names=('Non-churner','churner')))


## Random Forest

[공식 사이킷 런 문서](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) 를 참고하여 튜닝해보자.

반드시 알아야 할 것은 다음과 같다.

* n_estimators
* max_depth
* min_samples_split
* min_samples_leaf
* max_leaf_nodes


In [0]:
# 아래의 n_neighbors를 바꿔가며 관찰해보자.

clf = RandomForestClassifier(n_estimators = 30,
                            max_depth = None,
                            min_samples_split = 5,
                            min_samples_leaf = 1,
                            max_leaf_nodes = None)
clf.fit(X_train, Y_train)
Y_pred = clf.predict(X_test)
Y_pred_train = clf.predict(X_train)

print('트레이닝 7001명 중 실제 churner의 비율 : ',Y_train.mean())
print('트레이닝 7001명에 대해서 예측된 churner의 비율 :', Y_pred_train.mean())
print('모델의 트레이닝셋위의 정확도는 : {:.2f}'.format(clf.score(X_train, Y_train)*100))

print('--------------------------------------------')

print('테스트 2158명 중 실제 churner의 비율 : ',Y_test.mean())
print('테스트 2158명에 대해서 예측된 churner의 비율 :', Y_pred.mean())
print('모델의 테스트셋위의 정확도는 : {:.2f}'.format(clf.score(X_test, Y_test)*100))


print('---------------------------------------------')
print(confusion_matrix(Y_test,Y_pred).T)


print('---------------------------------------------')
print('precision : True라고 예측한 것 중 진짜 True 비율')
print('recall : 진짜 True 중 True라고 예측된 것의 비율')
print(classification_report(Y_test, Y_pred, target_names=('Non-churner','churner')))
